In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Data"

all-the-news-2-1.csv


In [ ]:
import pandas as pd

In [ ]:
for chunk in pd.read_csv('/content/drive/My Drive/Data/all-the-news-2-1.csv', chunksize=20000):
    df = pd.DataFrame(chunk)
    if df.empty == False:
        break
df.head(5)

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [ ]:
df.count()

date           20000
year           20000
month          20000
day            20000
author         14352
title          20000
article        19939
url            20000
section         8914
publication    20000
dtype: int64

In [ ]:
df['article'].isnull().values.sum()

61

In [ ]:
df_new = df[df['article'].notna()]

In [ ]:
df_new['article'].isnull().values.sum()

0

In [ ]:
df_new.count()

date           19939
year           19939
month          19939
day            19939
author         14322
title          19939
article        19939
url            19939
section         8861
publication    19939
dtype: int64

In [ ]:
df1 = df_new.drop(columns=['date', 'year', 'month', 'day', 'author', 'title', 'url', 'section', 'publication'])

In [ ]:
df1.head()

,article
0,"This post is part of Polyarchy, an independent..."
1,The Indianapolis Colts made Andrew Luck the h...
2,"DAVOS, Switzerland (Reuters) - U.S. President ..."
3,PARIS (Reuters) - Former French president Nico...
4,Paris Hilton arrived at LAX Wednesday dressed ...


In [ ]:
import torch


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.5 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum").to(device)

## Abstractive Summarization on the DataFrame

In [ ]:
df1['tokenized_article'] = df1.apply(lambda row: tokenizer(row['article'], truncation=True, padding="longest", return_tensors="pt").to(device), axis=1)

In [ ]:
df1.head()

,article,tokenized_article
0,"This post is part of Polyarchy, an independent...","[input_ids, attention_mask]"
1,The Indianapolis Colts made Andrew Luck the h...,"[input_ids, attention_mask]"
2,"DAVOS, Switzerland (Reuters) - U.S. President ...","[input_ids, attention_mask]"
3,PARIS (Reuters) - Former French president Nico...,"[input_ids, attention_mask]"
4,Paris Hilton arrived at LAX Wednesday dressed ...,"[input_ids, attention_mask]"


In [ ]:
import numpy as np
df_split = np.array_split(df1, 15)

In [ ]:
df_split1 = df_split[0]
df_split2 = df_split[1]
df_split3 = df_split[2]
df_split4 = df_split[3]
df_split5 = df_split[4]
df_split6 = df_split[5]
df_split7 = df_split[6]
df_split8 = df_split[7]
df_split9 = df_split[8]
df_split10 = df_split[9]
df_split11 = df_split[10]
df_split12 = df_split[11]
df_split13 = df_split[12]
df_split14 = df_split[13]
df_split15 = df_split[14]

In [ ]:
def translated_text(model, input_text):
    translated_text = model.generate(**input_text, max_length=512)
    return translated_text

In [ ]:
df_split6['translated_text'] = df_split6['tokenized_article'].apply(lambda x: translated_text(model, x).to(device))

In [ ]:
df_split5.head()

,article,tokenized_article,translated_text
5334,For a while it seemed that everyone was suing ...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(168, devi..."
5335,"Nobody does colorful, abstract, 3D art in moti...","[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(436, devi..."
5336,First-person essays and interviews with unique...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(139, devi..."
5337,10:14 AM PT -- It appears Kim made the trip wi...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(30166, de..."
5338,NEW DELHI (Reuters) - Ford Motor Co made a pro...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(168, devi..."


In [ ]:
df_split5['summary'] = df_split5.apply(lambda row: tokenizer.batch_decode(row['translated_text'], skip_special_tokens=True), axis=1)

In [ ]:
df_split5.head()

,article,tokenized_article,translated_text,summary
5334,For a while it seemed that everyone was suing ...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(168, devi...",[It’s been a long time coming.]
5335,"Nobody does colorful, abstract, 3D art in moti...","[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(436, devi...",[All images are copyrighted.]
5336,First-person essays and interviews with unique...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(139, devi...",[The Trump administration’s latest budget prop...
5337,10:14 AM PT -- It appears Kim made the trip wi...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(30166, de...",[Kanye West has left Los Angeles for Wyoming t...
5338,NEW DELHI (Reuters) - Ford Motor Co made a pro...,"[input_ids, attention_mask]","[[tensor(0, device='cuda:0'), tensor(168, devi...",[It’s been a long time coming.]


In [ ]:
df_split5['summary'][5388]

['Dozens of briefs have been filed with the US Supreme Court in support of lesbian, gay, bisexual and transgender (LGBT+) rights.']

In [ ]:
df_split5['article'][5388]

"NEW YORK (Thomson Reuters Foundation) - The mental and financial health of veterans and military families will suffer without explicit protections for LGBT+ Americans, advocates argued on Wednesday in a brief filed with the U.S. Supreme Court.  The nation’s highest court agreed in April to take on three LGBT+ workplace discrimination cases this October, marking the first indication of how its new conservative majority might handle LGBT+ rights.  A brief filed on behalf of military personnel, one of 49 filed on Wednesday bearing thousands of signatures, argued that the economic interests and dignity of LGBT+ military families  stationed in states without legal protections were endangered.  “Without Title VII protection, LGBT family members of service members can be denied the few available jobs in these areas and may have no income or have to travel long distances for work, both of which will burden their families,” it said.  At issue is whether LGBT+ people are covered by Title VII of

In [ ]:
df_split5_clean = df_split5.drop(columns=['tokenized_article', 'translated_text'])

In [ ]:
df_split5_clean = df_split5_clean.replace(r'\n',' ', regex=True) 

In [ ]:
df_split5_clean.head()

,article,summary
5334,For a while it seemed that everyone was suing ...,[It’s been a long time coming.]
5335,"Nobody does colorful, abstract, 3D art in moti...",[All images are copyrighted.]
5336,First-person essays and interviews with unique...,[The Trump administration’s latest budget prop...
5337,10:14 AM PT -- It appears Kim made the trip wi...,[Kanye West has left Los Angeles for Wyoming t...
5338,NEW DELHI (Reuters) - Ford Motor Co made a pro...,[It’s been a long time coming.]


In [ ]:
from google.colab import files


In [ ]:
df_split5_clean.to_csv('df_split5_clean.csv', encoding = 'utf-8-sig') 
files.download('df_split5_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls "/content/drive/My Drive/Train Data/"

df_split1_clean.csv  df_split3_clean.csv  df_split5_clean.csv
df_split2_clean.csv  df_split4_clean.csv  No-Summary


In [ ]:
df_batch1 = pd.read_csv('/content/drive/My Drive/Train Data/df_split1_clean.csv', index_col=0)
df_batch2 = pd.read_csv('/content/drive/My Drive/Train Data/df_split2_clean.csv', index_col=0)
df_batch3 = pd.read_csv('/content/drive/My Drive/Train Data/df_split3_clean.csv', index_col=0)
df_batch4 = pd.read_csv('/content/drive/My Drive/Train Data/df_split4_clean.csv', index_col=0)
df_batch5 = pd.read_csv('/content/drive/My Drive/Train Data/df_split5_clean.csv', index_col=0)

In [ ]:
dataframes = [df_batch1,df_batch2, df_batch3, df_batch4, df_batch5]

In [ ]:
df_merged = pd.concat(dataframes)

In [ ]:
df_merged.head()

,article,summary
0,"This post is part of Polyarchy, an independent...",['In the wake of a New York Times article sugg...
1,The Indianapolis Colts made Andrew Luck the h...,['The Indianapolis Colts made Andrew Luck the ...
2,"DAVOS, Switzerland (Reuters) - U.S. President ...","[""The special counsel investigating alleged Ru..."
3,PARIS (Reuters) - Former French president Nico...,['Sarkozy has retired twice from politics - th...
4,Paris Hilton arrived at LAX Wednesday dressed ...,['Paris Hilton arrived at LAX Wednesday dresse...


In [ ]:
df_merged.to_csv('df_merged.csv', encoding = 'utf-8-sig') 
files.download('df_merged.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>